# A test of Variational Monte Carlo on some Toy Models

Here we only deal with a toy model based on a Jastrow solution of Fermi-Hubbard model. We first validate this method in 1D, then apply this to the 2D case. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

## Fermi-Hubbard model

$$ H = -t \sum_{\langle ij \rangle} \sum_{\sigma} c_{i, \sigma}^{\dagger} c_{j, \sigma} + U \sum_{i} n_{i\uparrow} n_{i\downarrow}$$

(For 2D, there are more N.N. sites)

periodic boundary condition

## Noninteracting state (choose U=0 here instead of Hartree-Fock)

$$ c_{i\uparrow} = d_i $$

$$ c_{i\downarrow} = d_{i+L} $$

index $I$ running from 1 to 2L

简并会是问题吗？按现在的情况，确定了所选轨道的自旋，也就确定了采样出的x的自旋，即自旋不守恒的情况概率为零。但有简并的话，本征态应该是singlet/triplet


In [289]:
# system size (1D)
L = 8
N = 6
U = 0         # take tunneling strength as a unit
H1 = np.zeros((2*L,2*L))
for i in range(2*L-1):
    H1[i, i+1] = -1
H1[L-1, 0] = -1
H1[2*L-1, L] = -1
H1[L-1, L] = 0
H = H1 + H1.T
e, phi = np.linalg.eig(H)
phi_0 = phi[: , np.argsort(e)[:N]]
print(phi_0)
print(np.sum(np.sort(e)[:N]))

[[-3.53553391e-01  0.00000000e+00 -1.89712367e-02  0.00000000e+00
  -5.00000000e-01  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00  3.39884116e-01  0.00000000e+00
  -3.53553391e-01  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00  4.99639963e-01  0.00000000e+00
  -2.76642233e-16  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00  3.66713496e-01  0.00000000e+00
   3.53553391e-01  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00  1.89712367e-02  0.00000000e+00
   5.00000000e-01  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00 -3.39884116e-01  0.00000000e+00
   3.53553391e-01  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00 -4.99639963e-01  0.00000000e+00
  -2.11018414e-16  0.00000000e+00]
 [-3.53553391e-01  0.00000000e+00 -3.66713496e-01  0.00000000e+00
  -3.53553391e-01  0.00000000e+00]
 [ 0.00000000e+00 -3.53553391e-01  0.00000000e+00 -1.89712367e-02
   0.00000000e+00 -5.00000000e-01]
 [ 0.00000000e+00 -3.53553391e-01  0.00000000e+00  3.39884116e-01
   0.00000000e+00 -3.5355

## Jastrow-like Wavefunction

$$ |\psi \rangle = exp( -v \sum_{i} n_{i\uparrow} n_{i\downarrow}) |\phi _0 \rangle $$

In [298]:
# initialization
# v = np.random.randn()
v = 0
# x = np.random.choice(2*L, N, replace=False)
x_up = np.random.choice(L, np.int(N/2), replace=False)
x_down = np.random.choice(L, np.int(N/2), replace=False)
x = np.append(x_up, x_down+L)
site_empty = np.delete(np.arange(2*L), x)
site_double = np.intersect1d(x_up, x_down)
site_single = np.setdiff1d(x, site_double)
site_double_number = np.size(site_double)
print(x, x_up, site_empty, site_double, site_double_number)
print(phi_0[x])

[ 4  3  7 11 13 15] [4 3 7] [ 0  1  2  5  6  8  9 10 12 14] [3 7] 2
[[-0.35355339  0.          0.01897124  0.          0.5         0.        ]
 [-0.35355339  0.          0.3667135   0.          0.35355339  0.        ]
 [-0.35355339  0.         -0.3667135   0.         -0.35355339  0.        ]
 [ 0.         -0.35355339  0.          0.3667135   0.          0.35355339]
 [ 0.         -0.35355339  0.         -0.33988412  0.          0.35355339]
 [ 0.         -0.35355339  0.         -0.3667135   0.         -0.35355339]]


## Computation of expectations and variances of local operators $O$ (here for Hamiltonian $H$ especially)

$$ \langle O \rangle = \frac{\langle \Psi_J |O| \Psi_J \rangle}{\langle \Psi_J | \Psi_J \rangle} = \frac{ \sum_x |\langle \Psi_J| x\rangle |^2 \frac{\langle x |O| \Psi_J \rangle}{\langle x| \Psi_J \rangle}}{\sum_x |\langle \Psi_J| x\rangle |^2} = \sum_x p_J(x) O_L(x) $$

$$ H_{local} = \frac{\langle x |H| \Psi_J \rangle}{\langle x| \Psi_J \rangle} = \frac{ \sum_{x^{\prime}} \langle x |T| x^{\prime} \rangle \Psi_J(x^{\prime}) + U(x)\Psi_J(x)}{\Psi_J(x)} $$

$$ H^2_{local} = \frac{\langle x |(U+T)^2| \Psi_J \rangle}{\langle x| \Psi_J \rangle}  = \frac{ \sum_{x^{\prime \prime}} \langle x |T^2| x^{\prime \prime} \rangle \langle x^{\prime \prime}| \Psi_J \rangle + U(x) \langle x |T| \Psi_J \rangle + \sum_{x^{\prime}}\langle x |T| x^{\prime} \rangle U(x^{\prime}) \Psi_J(x^{\prime})+ U^2(x)\Psi_J(x)}{\ \Psi_J(x)} $$

$| x^{\prime} \rangle $ and $| x^{\prime \prime} \rangle $ are configurations differ from $| x \rangle $ with only one and two hopping and no change in spin.

So there are three terms about $| x^{\prime} \rangle $ and one term about $| x^{\prime \prime} \rangle $


In [305]:
# energy and variance
E = 0
sigma = 0

U = 0       # for test

x = np.append(x_up, x_down+L)
site_double = np.intersect1d(x_up, x_down)
double = np.size(site_double)

psi_x = np.linalg.det(phi_0[x])

jx = np.exp(-v * double)
psi_jx = psi_x * jx
ux = U * double

def onehop(x_in):
    result_hop = []

    # left hop
    for i in range(len(x_in)):
        if x_in[i] == 0:
            if L-1 in x_in:
                continue
            else:
                left = x_in.copy()
                left[i] = L-1
                result_hop.append(left)    

        elif x_in[i] - 1 in x_in:
            continue

        else:
            left = x_in.copy()
            left[i] = left[i] - 1
            result_hop.append(left)

    # right hop
    for i in range(len(x_in)):
        if x_in[i] == L-1:
            if 0 in x_in:
                continue
            else:
                right = x_in.copy()
                right[i] = 0
                result_hop.append(right)

        elif x_in[i] + 1 in x_in:
            continue

        else:
            right = x_in.copy()
            right[i] = right[i] + 1
            result_hop.append(right)

    return result_hop

onehop_up = onehop(x_up)
onehop_down = onehop(x_down)
twohop_up = []
for item in onehop_up:
    twohop_up = twohop_up + onehop(item)
twohop_down = []
for item in onehop_down:
    twohop_down = twohop_down + onehop(item)

# 只跳一次的话由以下两种情况构成：up一次down零次， up零次down一次。下面的式子按此排序
x_prime_up = onehop_up.copy()
for i in range(len(onehop_down)):
    x_prime_up.append(x_up)

x_prime_down = []
for i in range(len(onehop_up)):
    x_prime_down.append(x_down)
x_prime_down = x_prime_down + onehop_down

x_prime = [np.append(x_prime_up[i], x_prime_down[i]+L) for i in range(len(x_prime_up))]
site_double_prime = [np.intersect1d(x_prime_up[i], x_prime_down[i]) for i in range(len(x_prime))]
double_prime = np.array([np.size(site_double_prime[i]) for i in range(len(x_prime))])

# 跳两次的话由以下几种情况构成：up2down0， up0down2, 2*(up1down1) 下面的式子按此排序
x_prime2_up = twohop_up.copy()
for i in range(len(twohop_down)):
    x_prime2_up.append(x_up)

x_prime2_down = []
for i in range(len(twohop_up)):
    x_prime2_down.append(x_down)
x_prime2_down = x_prime2_down + twohop_down

len_onedown = len(onehop_down)
for item in onehop_up:
    for _ in range(2*len_onedown):
        x_prime2_up = x_prime2_up + [item]
    x_prime2_down = x_prime2_down + onehop_down + onehop_down

x_prime2 = [np.append(x_prime2_up[i], x_prime2_down[i]+L) for i in range(len(x_prime2_up))]
site_double_prime2 = [np.intersect1d(x_prime2_up[i], x_prime2_down[i]) for i in range(len(x_prime2))]
double_prime2 = np.array([np.size(site_double_prime2[i]) for i in range(len(x_prime2))])

psi_x_prime = np.array([np.linalg.det(phi_0[item]) for item in x_prime])
jx_prime = np.exp(-v * double_prime)
ux_prime = U * double_prime

E0 = (-np.sum(jx_prime * psi_x_prime) + ux*psi_jx) / psi_jx

psi_x_prime2 = np.array([np.linalg.det(phi_0[item]) for item in x_prime2])
jx_prime2 = np.exp(-v * double_prime2)

E02 = (U**2*psi_jx + np.sum(jx_prime2 * psi_x_prime2) - ux * np.sum(jx_prime * psi_x_prime) - np.sum(ux_prime * jx_prime * psi_x_prime)) / psi_jx

print(x_prime, x_prime2, jx_prime2, psi_x_prime, psi_jx)
print(len(x_prime), len(x_prime2))
print(E0, E02, E02 - E0**2)


# 由于T不改变自旋，故要分成两个子空间，并且注意边界条件！
# 顺序重要吗？ 由于认为输入的x是有序的，只要每次更改不变换位置，就不会有符号上的问题


[array([ 4,  2,  7, 11, 13, 15]), array([ 4,  3,  6, 11, 13, 15]), array([ 5,  3,  7, 11, 13, 15]), array([ 4,  3,  0, 11, 13, 15]), array([ 4,  3,  7, 10, 13, 15]), array([ 4,  3,  7, 11, 12, 15]), array([ 4,  3,  7, 11, 13, 14]), array([ 4,  3,  7, 12, 13, 15]), array([ 4,  3,  7, 11, 14, 15]), array([ 4,  3,  7, 11, 13,  8])] [array([ 3,  2,  7, 11, 13, 15]), array([ 4,  1,  7, 11, 13, 15]), array([ 4,  2,  6, 11, 13, 15]), array([ 5,  2,  7, 11, 13, 15]), array([ 4,  3,  7, 11, 13, 15]), array([ 4,  2,  0, 11, 13, 15]), array([ 4,  2,  6, 11, 13, 15]), array([ 4,  3,  5, 11, 13, 15]), array([ 5,  3,  6, 11, 13, 15]), array([ 4,  3,  7, 11, 13, 15]), array([ 4,  3,  7, 11, 13, 15]), array([ 5,  2,  7, 11, 13, 15]), array([ 5,  3,  6, 11, 13, 15]), array([ 6,  3,  7, 11, 13, 15]), array([ 5,  4,  7, 11, 13, 15]), array([ 5,  3,  0, 11, 13, 15]), array([ 4,  2,  0, 11, 13, 15]), array([ 4,  3,  7, 11, 13, 15]), array([ 5,  3,  0, 11, 13, 15]), array([ 4,  3,  1, 11, 13, 15]), array([ 

## Update of determinant $|\frac{\phi_0(x^\prime)}{\phi_0(x)}|^2$ :

$x^\prime$ differs from x with only one index, x ranges from 0 to 2L-1 (spin up for 0 ~ L-1) 

### Some details: 

Notation:  $ \tilde{U} = \phi_0(x)   $



## Update of Jastrow factor:

Since there are only diagnoal terms in Gutzwiller factor, we only need to keep track with doubly occupied sites, which are nonvanishing in the sum on the exponential term.


In [79]:
# randomly choose a new configuration
index_change = np.random.randint(N)
index_new = np.random.randint(2*L-1-N)
x_new = x.copy()
x_new[index_change] = site_empty[index_new]
# print(x, site_double, index_change, index_new, x_new)

# update of Jastrow factor
site_double_number_new = site_double_number

if x[index_change]%L in site_double:
    arg_delete = np.argwhere(site_double, x[index_change]%L)[0]
    site_double_new = np.delete(site_double, site_double[arg_delete])
    site_double_number_new += (-1)

if x_new[index_change]%L in site_single:
    arg_append = np.argwhere(site_double, x[index_change]%L)[0]
    site_double_new = np.delete(site_double, site_double[arg_delete])
    site_double_number_new += (-1)


# update of determinant

first_matrix = phi_0[x]            # TODO: dealing with matrix element = 0
first_matrix_inv = np.linalg.inv(first_matrix)
first_matrix_det = np.linalg.det(first_matrix)
delta_v = U[np.ix_([j],v[0:i+1])] - U[np.ix_([j0],v[0:i+1])]
detRatio = 1 + np.dot(first_matrix_inv[:,-1],delta_v[0,:])

# Recept in Markov chain
r = 0.2
if np.random.rand() < np.min(1, r):
    pass



[1 6]
[1 6] [] 0 1 [2 6]


In [ ]:
# energy



In [ ]:
# variation


## Place for simple test

In [307]:
# test
8 % 3 

2